In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from demand_restoration import *
from preABT import *
from forecasting import *
from reconcilation import *
from hybridization import *
from disaccumulation import *

In [2]:
DR_PARAMETERS = {
    'DR_OBS_NUM' : 30,
    'DR_LIFECYCLE_MARGIN' : 7,
    'DR_PERIOD_LENGTH' : 30,
    'DEF_INV_TRSHD' : 1000,
    'DEF_QTY_TRSHD' : 10,
    'MIN_SALES_QTY_DAY' : 0,
    'MIN_PROLONG_HIST_MONTH' : 3,
    'MAX_PROLONG_HIST_MONTH' : 24,
    'HIGH_TURNOVER_TRSHD' : 1000,
    'MIN_ND_DAYS' : 1
}

TGT_VAR_CONFIG = {
    'tgt_type' : 'pos',
    'value_src' : 'SALES_QTY',
    'act_flag' : 1,
    'dr_scen' : 3,
    'link_with_stock' : 1,
    'link_with_promo' : 0,
    'link_with_price' : 1,
    'vf_product_lvl' : 7,
    'vf_location_lvl' : 1,
    'vf_customer_lvl' : 5,
    'vf_distr_channel_lvl' : 1,
    'vf_time_lvl' : 'MONTH',
    'ml_product_lvl' : 7,
    'ml_location_lvl' : 5,
    'ml_customer_lvl' : 4,
    'ml_distr_channel_lvl' : 1,
    'ml_time_lvl' : 'WEEK'
}

SEASONAL_FLAG_CONFIG = {
    'EA' : 1,
    'NY' : 1,
    'BA' : 1,
    'XX' : 1,
    'YY' : 1
}

SEASONAL_FLAG_CONFIG = pd.DataFrame([SEASONAL_FLAG_CONFIG]).T.reset_index().rename(columns={'index':'PRODUCT_ATTR_NAME', 0:'PRODUCT_ATTR_VALUE'})

IB_MAX_DT = datetime.datetime(2100, 1, 1)
IB_UPDATE_HISTORY_DEPTH = 30
IB_HIST_START_DT = datetime.datetime(2018, 4, 1)
IB_HIST_END_DT = datetime.datetime(2020, 3, 19)
HIGH_TURNOVER_TRSHD = 100


In [3]:
assort_matrix = pd.read_csv('./new_data/DFR_ASSORT_MATRIX.csv')
price = pd.read_csv('./new_data/DFR_PRICE.csv')
STOCK = pd.read_csv('./new_data/DFR_STOCK.csv')
SALES = pd.read_csv('./new_data/DFR_SALES.csv')
PROMO = pd.read_csv('./new_data/DFR_PROMO.csv')

LOCATION = pd.read_csv('./new_data/DFR_LOCATION.csv')
location_life = pd.read_csv('./new_data/DFR_LOCATION_LIFE.csv')
LOCATION_ATTR = pd.read_csv('./new_data/DFR_LOCATION_ATTR.csv')


PRODUCT = pd.read_csv('./new_data/DFR_PRODUCT.csv')
product_life = pd.read_csv('./new_data/DFR_PRODUCT_LIFE.csv')
PRODUCT_ATTR = pd.read_csv('./new_data/DFR_PRODUCT_ATTR.csv')

hierarchies = {
    'PRODUCT' : PRODUCT,
    'LOCATION' : LOCATION
}


LOCATION.columns = [i.upper() for i in list(LOCATION.columns)]
SALES.columns = [i.upper() for i in list(SALES.columns)]
STOCK.columns = [i.upper() for i in list(STOCK.columns)]

SALES.PERIOD_DT = pd.to_datetime(SALES.PERIOD_DT)

In [14]:
def execute_pipeline():
    df = demand_restore(DR_PARAMETERS, TGT_VAR_CONFIG, IB_MAX_DT, IB_UPDATE_HISTORY_DEPTH, IB_HIST_START_DT,
        IB_HIST_END_DT, hierarchies, STOCK, PROMO, SALES,
        HIGH_TURNOVER_TRSHD, PRODUCT_ATTR, SEASONAL_FLAG_CONFIG)
    abt = preABT(df)
    ML_FORECAST = forecast_ml(abt)
    VF_FORECAST = forecast_ts(abt)
    VF_FORECAST.to_csv('VF_FORECAST.csv', sep = ';', index = False)
    reconciled_forecast = reconcilation_algorithm(VF_FORECAST, ML_FORECAST)
    hybridizated_forecast = hybridization_algorithm(reconciled_forecast)
    Dis = Disaccumulation(hybridizated_forecast, 'D')
    data_splitted = Dis.split_forecasts()
    return Dis.data_splitted.drop_duplicates(['PRODUCT_ID', 'LOCATION_ID', 'PERIOD_DT'])

In [20]:
df = execute_pipeline()

IntProgress(value=0, max=24)

[0]	train-rmse:69.57163	validation-rmse:121.16920
[5]	train-rmse:56.43337	validation-rmse:118.91499
[10]	train-rmse:51.34032	validation-rmse:119.24250
[14]	train-rmse:48.06964	validation-rmse:119.84444


  0%|          | 0/24238 [00:00<?, ?it/s]

In [17]:
df.to_csv('final_pipeline_results.csv', sep = ';', index = False)

In [21]:
df

,PERIOD_DT,PERIOD_END_DT,PRODUCT_ID,LOCATION_ID,CUSTOMER_ID,ASSORTMENT_TYPE,DEMAND_TYPE,SEGMENT_NAME,VF_FORECAST_VALUE,ML_FORECAST_VALUE,HYBRID_FORECAST_VALUE,FORECAST_SOURCE
0,2018-04-01,2018-04-01,8000054507,5637144625,56371,old,promo,Other,0.339290,9.806688,9.806688,ml
1,2018-04-02,2018-04-02,8000054507,5637144625,56371,old,promo,Other,0.339290,9.806688,9.806688,ml
2,2018-04-03,2018-04-03,8000054507,5637144625,56371,old,promo,Other,0.339290,9.806688,9.806688,ml
3,2018-04-04,2018-04-04,8000054507,5637144625,56371,old,promo,Other,0.339290,9.806688,9.806688,ml
4,2018-04-05,2018-04-05,8000054507,5637144625,56371,old,promo,Other,0.339290,9.806688,9.806688,ml
...,...,...,...,...,...,...,...,...,...,...,...,...
98650,2020-03-20,2020-03-20,9000400501,9000136500,90001,new,regular,Other,0.256032,1.905398,1.905398,ml
98651,2020-03-21,2020-03-21,9000400501,9000136500,90001,new,regular,Other,0.256032,1.905398,1.905398,ml
98652,2020-03-19,2020-03-19,9000400501,9000136500,90001,old,regular,Other,0.256032,1.905398,1.905398,ml
98653,2020-03-20,2020-03-20,9000400501,9000136500,90001,old,regular,Other,0.256032,1.905398,1.905398,ml
